In [1]:
import glob

import numpy as np

from eminus import Atoms, read, SCF
from eminus.extras import view
from eminus.localizer import get_wannier, wannier_cost
from eminus.orbitals import FLO, WO
from eminus.tools import orbital_center

In [2]:
# Run an initial calculation for methane
atoms = Atoms(*read('CH4.xyz'), ecut=15, center=True)
scf = SCF(atoms)
scf.run();

XYZ file comment: "Experimental geometry from CCCBDB: https://cccbdb.nist.gov/exp2x.asp?casno=74828&charge=0"
Start auto minimization...
Method  Iteration  Etot [Eh]    dEtot [Eh]   |Gradient|   
pccg           1   +25.949780   
pccg           2   +7.616602    +1.8333e+01  [+4.53e+04]  
pccg           3   -0.323833    +7.9404e+00  [+3.05e+03]  
pccg           4   -4.387684    +4.0639e+00  [+4.99e+02]  
pccg           5   -6.286317    +1.8986e+00  [+9.18e+01]  
pccg           6   -6.933588    +6.4727e-01  [+1.47e+01]  
pccg           7   -7.302068    +3.6848e-01  [+4.84e+00]  
pccg           8   -7.567839    +2.6577e-01  [+5.72e+00]  
pccg           9   -7.789691    +2.2185e-01  [+2.44e+00]  
pccg          10   -7.858153    +6.8461e-02  [+4.57e-01]  
pccg          11   -7.875782    +1.7629e-02  [+1.19e-01]  
pccg          12   -7.879610    +3.8278e-03  [+2.71e-02]  
pccg          13   -7.880573    +9.6318e-04  [+6.99e-03]  
pccg          14   -7.880808    +2.3452e-04  [+1.72e-03]  
pccg

In [3]:
# Calculate the FLOs to have pre-localized orbitals
flo = FLO(scf)

/home/wangenau/.local/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [4]:
# Do the Wannier localization
# The resulting orbitals are equivalent to Foster-Boys orbitals, but with periodic boundary conditions
wannier = get_wannier(atoms, flo)

Wannier localizer converged after 473 iterations.


In [5]:
# Compare the initial FLO spreads to the Wannier spreads
flo_spreads = wannier_cost(atoms, flo)
print(f'\nFLO spreads = {flo_spreads}')
print(f'FLO spread = {np.sum(flo_spreads)}')

Costs:
[4.09999183 3.86810969 4.35884166 3.36608968]

FLO spreads = [[4.09999183 3.86810969 4.35884166 3.36608968]]
FLO spread = 15.693032868299042


In [6]:
# The Wannier orbitals are a bit more localized, and all orbitals are evenly localized
# (Note that the FLO routine just uses a crude PyCOM guess and does not optimize the FODs.)
wannier_spreads = wannier_cost(atoms, wannier)
print(f'Wannier spreads = {wannier_spreads}')
print(f'Wannier spread = {np.sum(wannier_spreads)}')

Costs:
[2.68198219 2.68212498 2.68202201 2.6821521 ]
Wannier spreads = [[2.68198219 2.68212498 2.68202201 2.6821521 ]]
Wannier spread = 10.728281273808477


In [ ]:
# All of the above can be done with one function call, also save the orbitals
WO(scf, write_cubes=True);

Wannier localizer converged after 452 iterations.
Write CH4_WO_0.cube...
Write CH4_WO_1.cube...


In [ ]:
# Display the orbitals from the cube files
view(glob.glob('*.cube'));

In [ ]:
# One could calculate the center of masses from the Wannier orbitals...
coms = orbital_center(atoms, wannier)
view(atoms, coms)

In [ ]:
# ...and use them as an initial guess for a new set of FLOs
flo = FLO(scf, fods=coms)
flo_spreads = wannier_cost(atoms, flo)
print(f'New FLO spreads = {flo_spreads}')
print(f'New FLO spread = {np.sum(flo_spreads)}')